In [1]:
#GROVER'S ALGORITHMS
my_list = [1,3,5,2,4,9,5,8,0,7,6]

In [10]:
# Classical Approach
def the_oracle(my_input):
    winner = 7
    if my_input is winner:
        response = True
    else:
        response = False
    return response

In [11]:
for index, trial_number in enumerate(my_list):
    if the_oracle(trial_number) is True:
        print('Winner found at index %i'%index)
        print('%i calls to the Oracle used' %(index+1))
        break 

Winner found at index 9
10 calls to the Oracle used


In [12]:
from qiskit import *
import matplotlib.pyplot as plt
import numpy as np

In [13]:
# define oracle circuit or operator
oracle = QuantumCircuit(2, name='oracle')
oracle.cz(0,1)
oracle.to_gate() #make the oracle into its own gate
oracle.draw()

q_0: ─■─
      │ 
q_1: ─■─

In [15]:
# check the oracle is doing the right thing
backend = Aer.get_backend('statevector_simulator')
grover_circ = QuantumCircuit(2,2)
grover_circ.h([0,1]) # this prepares our superposition states
grover_circ.append(oracle,[0,1])
grover_circ.draw()


┌───┐┌─────────┐
q_0: ┤ H ├┤0        ├
     ├───┤│  oracle │
q_1: ┤ H ├┤1        ├
     └───┘└─────────┘
c: 2/════════════════

In [17]:
job = execute(grover_circ,backend)
result = job.result()

In [18]:
sv = result.get_statevector()
np.round(sv,2)

array([ 0.5+0.j,  0.5+0.j,  0.5+0.j, -0.5+0.j])

In [20]:
# now we need reflection operator
reflection = QuantumCircuit(2,name='reflection')
reflection.h([0,1])
reflection.z([0,1])
reflection.cz(0,1)
reflection.h([0,1])
reflection.to_gate()
reflection.draw()

┌───┐┌───┐   ┌───┐
q_0: ┤ H ├┤ Z ├─■─┤ H ├
     ├───┤├───┤ │ ├───┤
q_1: ┤ H ├┤ Z ├─■─┤ H ├
     └───┘└───┘   └───┘

In [23]:
backend = Aer.get_backend('qasm_simulator')
grover_circ = QuantumCircuit(2,2)
grover_circ.h([0,1])
grover_circ.append(oracle,([0,1]))
grover_circ.append(reflection,[0,1])
grover_circ.measure([0,1],[0,1])


In [24]:
grover_circ.draw()

┌───┐┌─────────┐┌─────────────┐┌─┐   
q_0: ┤ H ├┤0        ├┤0            ├┤M├───
     ├───┤│  oracle ││  reflection │└╥┘┌─┐
q_1: ┤ H ├┤1        ├┤1            ├─╫─┤M├
     └───┘└─────────┘└─────────────┘ ║ └╥┘
c: 2/════════════════════════════════╩══╩═
                                     0  1

In [26]:
job = execute(grover_circ,backend,shots =1)
result = job.result()
result.get_counts()

{'11': 1}